### Transfer Learning
Here, I'll be using a [dataset of cat and dog photos](https://www.kaggle.com/c/dogs-vs-cats) available on Kaggle.

In [1]:
# importing all the necessary modules
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data_dir = './../data/dogs-vs-cats/'

# defining transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# defining the train data and test data with their transforms
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

# defining the trainloader and testloader
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [3]:
# to use GPU if it's available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
model = models.densenet121(pretrained=True)

# freeze the parameters
for param in model.parameters():
    param.requires_grad = False

# defining a model
model.classifier = nn.Sequential(nn.Linear(1024, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 2),
                                 nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()

# defining an opotimizer
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

# moving the model to work on GPU if available
model.to(device);

C:\Users\96653\Anaconda3\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [5]:
epochs = 1
steps = 0
running_loss = 0
print_every = 10

for epoch in range(epochs):
    # loop through the trainloader generator
    for images, labels in trainloader:
        #counting the steps 
        steps += 1
        # move the images and labels to the GPU
        images, labels = images.to(device), labels.to(device)
        # zero the grad
        optimizer.zero_grad()
        
        # train the model
        logps = model.forward(images)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        
        # acculmating the running loss
        running_loss += loss.item()
        
        # if steps are 10, perform the test
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            # test mode
            model.eval()
            with torch.no_grad():
                # loop through the estloader generator
                for images, labels in testloader:
                    # move the images and labels to the GPU
                    images, labels = images.to(device), labels.to(device)
                    # test the model
                    logps = model.forward(images)
                    loss = criterion(logps, labels)
                    test_loss += loss.item()
                    
                    # calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))
                    
            print(f'Epoch {epoch+1}/{epochs}.. '
                  f'Train loss: {running_loss/print_every:.2f}.. '
                  f'Test loss: {test_loss/len(testloader):.2f}.. '
                  f'Test accuracy: {accuracy/len(testloader):.2f}')
            running_loss = 0
            model.train()

Epoch 1/1.. Train loss: 0.16.. Test loss: 0.07.. Test accuracy: 0.97
Epoch 1/1.. Train loss: 0.20.. Test loss: 0.05.. Test accuracy: 0.98
Epoch 1/1.. Train loss: 0.15.. Test loss: 0.05.. Test accuracy: 0.98
